## Import Libraries

In [1]:
from utils import ArmHandler
from time import sleep
import numpy as np
import cv2
from random import randint

SDK_VERSION: 1.13.8


In [15]:
arm = ArmHandler()

ROBOT_IP: 192.168.1.155, VERSION: v2.2.0, PROTOCOL: V1, DETAIL: 6,9,LI1006,DL1000,v2.2.0, TYPE1300: [0, 0]
change protocol identifier to 3


## Define Functions

In [4]:
def mouse_callback(event: int, x: int, y: int, flags, param):
    """
    Callback function for mouse events, Set the global variable point to the current pixel coordinates, (inverted Px,Py)
    """
    if event == cv2.EVENT_LBUTTONDOWN:
        global point
        point = (y, x)


def add_rows(matrix, row):
    if matrix.size == 0:
        return np.array(row).reshape(1, -1)
    else:
        return np.vstack((matrix, np.array(row)))

## Start Video Capture

In [11]:
cap = cv2.VideoCapture(1)
cap.set(3, 640)
cap.set(4, 480)
cv2.namedWindow("Video")
cv2.setMouseCallback("Video", mouse_callback)

## Collect Data Points

In [8]:
f = open("calibration_data/points.txt", "a")
for i in range(25):
    arm.pickup_and_wait()
    bot_x, bot_y = (randint(215, 275), randint(0, 300))
    arm.place_block_on(bot_x, bot_y, z=27)

    point: tuple[int, int] | None = None

    while not point:
        ret, frame = cap.read()
        cv2.imshow("Video", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    if not point:  # If user exits the loop, point maybe None
        break

    print(f"Robot: {(bot_x, bot_y)}, Pixel: {point}")
    f.write(f"{(bot_x, bot_y)},{point}\n")
f.close()

Robot: (259, 221), Pixel: (160, 471)
Robot: (245, 235), Pixel: (137, 490)
Robot: (266, 277), Pixel: (168, 553)
Robot: (245, 159), Pixel: (142, 375)
Robot: (273, 139), Pixel: (187, 345)
Robot: (227, 251), Pixel: (111, 513)
Robot: (265, 203), Pixel: (172, 444)
Robot: (266, 148), Pixel: (177, 359)
Robot: (232, 110), Pixel: (127, 300)
Robot: (267, 111), Pixel: (178, 305)
Robot: (252, 154), Pixel: (155, 368)
Robot: (246, 204), Pixel: (144, 443)
Robot: (261, 38), Pixel: (174, 192)
Robot: (244, 5), Pixel: (154, 144)
Robot: (252, 111), Pixel: (158, 301)
Robot: (269, 164), Pixel: (181, 382)
Robot: (236, 14), Pixel: (138, 156)
Robot: (258, 219), Pixel: (160, 468)
Robot: (215, 50), Pixel: (106, 210)
Robot: (264, 47), Pixel: (182, 204)
Robot: (232, 249), Pixel: (121, 511)
Robot: (229, 164), Pixel: (119, 380)
Robot: (217, 138), Pixel: (103, 338)
Robot: (223, 30), Pixel: (119, 180)


## Process Data, and find the transformation matrix

In [12]:
Rx = np.array([])
Ry = np.array([])
Px = np.array([])
Py = np.array([])

with open("calibration_data/points.txt") as f:
    for line in set(f.readlines()):
        bot_coords, pixel_coords = eval(line)
        Rx = add_rows(Rx, [bot_coords[0]])
        Ry = add_rows(Ry, [bot_coords[1]])
        Px = add_rows(Px, [pixel_coords[0]])  # ...(Px,[pixel_coords[0],1]) - old using no rotation assumption
        Py = add_rows(Py, [pixel_coords[1]])  # ...(Py,[pixel_coords[1],1]) - old using no rotation assumption
print(Rx.size)
R = np.concatenate([Rx.T, Ry.T, np.ones(Rx.size).T.reshape(1, -1)], axis=0)
P = np.concatenate([Px.T, Py.T, np.ones(Px.size).T.reshape(1, -1)], axis=0)
W = R @ np.linalg.pinv(P)
print(W)

mx1, my1, c1, mx2, my2, c2 = map(lambda x: round(x, 10), (W[0, 0], W[0, 1], W[0, 2], W[1, 0], W[1, 1], W[1, 2]))

print(f"mx1,my1,c1 = {mx1},{my1},{c1}")
print(f"mx2,my2,c2 = {mx2},{my2},{c2}")

# old using no rotation assumption

# m1, c1 = np.linalg.pinv(Px) @ Rx
# m2, c2 = np.linalg.pinv(Py) @ Ry
# 
# m1, c1 = round(m1[0], 16), round(c1[0], 16)
# m2, c2 = round(m2[0], 16), round(c2[0], 16)
# print(f"m1,c1 = {m1},{c1}")
# print(f"m2,c2 = {m2},{c2}")

75
[[ 6.69519094e-01  2.62044828e-02  1.38643046e+02]
 [-2.51056056e-02  6.59144881e-01 -8.48399853e+01]
 [-6.50521303e-19  6.77626358e-19  1.00000000e+00]]
mx1,my1,cx1 = 0.6695190945,0.0262044828,138.6430459748
mx2,my2,cx2 = -0.0251056056,0.6591448809,-84.8399852824


In [13]:
# Hand Calculated (2 blocks):
# m1, c1 = 0.7251, 179.5495
# m2, c2 = 0.6383, -82.3553
# 10 blocks:
# m1, c1 = 0.71108,179.96984
# m2,c2 = 0.63734, -87.31099
# 39 blocks:
# m1,c1 = 0.56171, 197.02872
# m2,c2 = 0.64017,-85.75084
# 50 blocks:
# m1, c1 = 0.54665, 198.17924
# m2, c2 = 0.64072, -86.07211
# Reposition - 50 blocks
# m1, c1 = 0.6519, 150.83156
# m2,c2 = 0.64737, -84.42875
# Reposition - 50 blocks
# m1, c1 = 0.6456, 151.74737
# m2, c2 = 0.65968, -88.87191
# Same - 75 blocks
# m1, c1 = 0.6515, 150.83251
# m2, c2 = 0.65973, -88.76217


def pixel_to_world(px, py):
    return mx1 * px + my1 * py + c1, mx2 * px + my2 * py + c2  # m1 * px + c1, m2 * py + c2 - old using no rotation assumption

In [16]:
point = None
while True:
    if point is not None:
        rx, ry = pixel_to_world(point[0], point[1])
        print("Pixel: ", point)
        print(f"World: ({rx}, {ry})")
        if arm.pickup_block_from(rx, ry, z=25):
            arm.place_block_on()
        point = None

    ret, frame = cap.read()
    cv2.imshow("Video", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

Pixel:  (171, 166)
World: (257.4807552791, 20.285006389399996)
Pixel:  (176, 167)
World: (260.8545552344, 20.818623242300006)


In [17]:
arm.disconnect()
cap.release()
cv2.destroyAllWindows()